<a href="https://colab.research.google.com/github/yasuhiroinoue/AppMathSci/blob/main/%E5%BF%9C%E7%94%A8%E6%95%B0%E7%90%86(2021_11_22).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**応用数理科学（第8回）**##
今回から数回にわたり、時系列解析を学んでいこう。
自己回帰型モデル、状態推定モデルのいくつか有名なモデルを解説する。

AR, MA, ARMA, ARIMA
執筆中